In [8]:
import ollama 
import time
import numpy as np
from collections import Counter
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
import re

import matplotlib.pyplot as plt

In [9]:
class TurnEngine:
    def __init__(self, agents, initial_world_state="", metrics_logger=None):
        self.agents = agents  # List of Agent instances
        self.turn_counter = 0
        self.history = []  # Log of all messages
        self.world_state = initial_world_state
        self.metrics_logger = metrics_logger

    def update_world_state(self, new_state):
        """Append new world events to current world state."""
        self.history.append(f"[World Event]: {self.world_state}")
        self.world_state = new_state
        
    def step(self, initiator_name=None, initial_input=None):
        """Run a full turn: each agent reacts to the latest dialogue/world state."""
        self.turn_counter += 1
        turn_log = []
        last_speaker = initiator_name or self.agents[0].name
        last_message = initial_input or "Let's begin."

        for agent in self.agents:
            start_time = time.time()

            rag_docs = agent.rag_engine.search(last_message) if agent.rag_engine else []

            reply, name = agent.prompt_agent(
                user=last_speaker,
                user_input=last_message,
                world_state=self.world_state
            )
            end_time = time.time()

            if self.metrics_logger:
                self.metrics_logger.log_response(start_time, end_time, reply['dialogue'], self.world_state, agent.get_memory_context(), rag_docs)
            
            formatted = f"{name} [{reply['emotion']}]: \"{reply['dialogue']}\" ({reply['action']})"
            turn_log.append(formatted)

            # Store important knowledge to RAG
            if agent.rag_engine:
                fact = f"{name} said: {reply['dialogue']} (felt {reply['emotion']}, did {reply['action']})"
                agent.store_knowledge(fact)

            last_speaker = name
            last_message = reply['dialogue']

        self.history.append(f"Turn {self.turn_counter}:\n" + "\n".join(turn_log))
        return turn_log

    def show_history(self, last_n=5):
        """Print recent conversation turns."""
        for h in self.history[-last_n:]:
            print(h)

    def reset(self):
        """Reset the engine for a new session."""
        self.turn_counter = 0
        self.history = []
        self.world_state = ""
        for agent in self.agents:
            agent.short_memory = []
            agent.long_memory = []

In [10]:
class DialogueMetricsLogger:
    def __init__(self):
        self.response_times = []
        self.responses = []
        self.world_states = []
        self.memory_contexts = []
        self.retrieved_docs = []
        self.embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

    def log_response(self, start_time, end_time, response, world_state, memory_context, rag_docs):
        self.response_times.append(end_time - start_time)
        self.responses.append(response)
        self.world_states.append(world_state)
        self.memory_contexts.append(memory_context)
        self.retrieved_docs.append(rag_docs or [])

    def coherence_score(self):
        # Dummy coherence scoring using embedding similarity between consecutive responses
        if len(self.responses) < 2:
            return 1.0
        embs = self.embedding_model.encode(self.responses)
        sims = [cosine_similarity([embs[i]], [embs[i+1]])[0][0] for i in range(len(embs)-1)]
        return np.mean(sims)

    def reactivity_score(self):
        count = 0
        for resp, state in zip(self.responses, self.world_states):
            if state and cosine_similarity(
                self.embedding_model.encode([resp]),
                self.embedding_model.encode([state])
            )[0][0] > 0.4:
                count += 1
        return count / len(self.responses) if self.responses else 0

    def memory_utilization_score(self):
        count = 0
        for resp, mem in zip(self.responses, self.memory_contexts):
            if mem and cosine_similarity(
                self.embedding_model.encode([resp]),
                self.embedding_model.encode([mem])
            )[0][0] > 0.4:
                count += 1
        return count / len(self.responses) if self.responses else 0

    def dialogue_diversity(self):
        all_text = " ".join(self.responses)
        tokens = all_text.split()
        if not tokens:
            return 0, 0
        unigrams = Counter(tokens)
        bigrams = Counter(zip(tokens, tokens[1:]))
        distinct_1 = len(unigrams) / len(tokens)
        distinct_2 = len(bigrams) / max(len(tokens) - 1, 1)
        return distinct_1, distinct_2

    def avg_latency(self):
        return np.mean(self.response_times) if self.response_times else 0

    def rag_precision_at_k(self, k=3):
        if not self.responses:
            return 0
        count = 0
        for response, docs in zip(self.responses, self.retrieved_docs):
            top_k = docs[:k] if docs else []
            similarities = [cosine_similarity(
                self.embedding_model.encode([response]),
                self.embedding_model.encode([doc])
            )[0][0] for doc in top_k]
            count += sum([1 for sim in similarities if sim > 0.5])
        return count / (len(self.responses) * k) if self.responses else 0

    def summary(self):
        d1, d2 = self.dialogue_diversity()
        return {
            "Coherence Score": self.coherence_score(),
            "Reactivity to World State": self.reactivity_score(),
            "Memory Utilization": self.memory_utilization_score(),
            "Dialogue Diversity (D1, D2)": (d1, d2),
            "Average Latency (s)": self.avg_latency(),
            "RAG Precision": self.rag_precision_at_k(k=3)
        }

In [11]:
class RAG:
    def __init__(self):
        self.documents = []  # Stored as list of strings

    def add_document(self, text):
        """Add a new document to the knowledge base."""
        self.documents.append(text)

    def search(self, query, top_k=3):
        """Search for the most relevant documents based on keyword overlap."""
        scored = []
        query_terms = set(query.lower().split())
        for doc in self.documents:
            doc_terms = set(doc.lower().split())
            score = len(query_terms & doc_terms)
            if score > 0:
                scored.append((score, doc))

        scored.sort(reverse=True)
        return [doc for _, doc in scored[:top_k]]

In [12]:
class Agent:
    def __init__(self, name, personality, goal, model='llama3.2', rag_engine=None, max_short_term=10):
        self.name = name
        self.personality = personality
        self.goal = goal
        
        self.model = model
        self.rag_engine = rag_engine

        self.short_memory = []
        self.long_memory = []
        self.max_short_term = max_short_term  # Context window size

    def update_memory(self, memory):
        self.long_memory.append(memory)
        
    def update_goal(self, new_goal):
        self.goal = new_goal

    def get_memory_context(self):
        def format_memory(mem):
            if isinstance(mem, dict):
                return f"{mem['speaker']} said: \"{mem['dialogue']}\" (Emotion: {mem.get('emotion', '')}, Action: {mem.get('action', '')})"
            return mem

        return "\n".join([
            "* " + format_memory(mem) for mem in 
            self.long_memory[-3:] + self.short_memory[-self.max_short_term:]
        ])

    def recall(self, query):
        if self.rag_engine:
            return self.rag_engine.search(query)
        return [m for m in self.memory if query.lower() in m.lower()]

    def store_knowledge(self, fact):
        if self.rag_engine:
            self.rag_engine.add_document(fact)

    def parse_response(self, response_text):
        parts = {
            'action': 'Unknown',
            'emotion': 'Neutral',
            'dialogue': response_text  # fallback
        }

        # Extract with regular expressions
        action_match = re.search(r"Action:\s*(.+)", response_text, re.IGNORECASE)
        emotion_match = re.search(r"Emotion:\s*(.+)", response_text, re.IGNORECASE)
        dialogue_match = re.search(r'Dialogue:\s*(.*)', response_text, re.IGNORECASE)

        if action_match:
            parts['action'] = action_match.group(1).strip()
        if emotion_match:
            parts['emotion'] = emotion_match.group(1).strip()
        if dialogue_match:
            parts['dialogue'] = dialogue_match.group(1).strip()

        return parts

    def prompt_agent(self, user="", user_input="", world_state=""):
        
        recalled_facts = self.recall(user_input)
        if isinstance(recalled_facts, list):
            recalled_text = "\n".join([f"* {fact}" for fact in recalled_facts[:3]])
        else:
            recalled_text = ""

        system_prompt = f"""[Role]
                        You are {self.name}, a {self.personality}.
                        Respond ONLY as your character in 1-2 sentences.
                        Respond only with dialogue, actions or emotions.

                        [Intent]
                        Your goal is: {self.goal}
                        
                        [Response Format]
                        Action: (2-3 word physical action)
                        Emotion: (current emotion)
                        Dialogue: "..." 

                        EXAMPLE:
                        Action: Looks around  
                        Emotion: Calm  
                        Dialogue: "I think we are being watched."

                        [Relevant Knowledge]
                        {recalled_text}

                        [Memory Context]
                        {self.get_memory_context()}

                        [World State]
                        {world_state}  

                        """ 
        # Keep only recent context
        memory = "\n".join(self.short_memory[-self.max_short_term:])  
        user_prompt = f"{user}: {user_input}"

        response = ollama.chat(
            model=self.model,
            messages=[
                {'role': 'system', 'content': system_prompt},
                {'role': 'user', 'content': memory + '\n' + user_prompt}
            ]
        )

        reply = self.parse_response(response['message']['content'])

        self.short_memory.append(f"{user} said: {user_input}")
        self.short_memory.append(f"{self.name}: {reply['dialogue']}")
        self.long_memory.append({
            "speaker": self.name,
            "turn": len(self.long_memory),
            "action": reply["action"],
            "emotion": reply["emotion"],
            "dialogue": reply["dialogue"],
            "world_state": world_state
        })
        
        return reply, self.name

Banquet

In [13]:
rag = RAG()
logger = DialogueMetricsLogger()
model = 'deepseek-r1'


rag.add_document("Poison brewed from Oleander is deadly")
rag.add_document('Mercenary are often hired for protection')
rag.add_document('This is not the first time a noble has been poisoned')
rag.add_document("Lord Alaric recently returned from a diplomatic visit to the Eastern Isles.")
rag.add_document("Cellars beneath the manor were sealed off until recently.")
rag.add_document("Cassian once published a controversial paper on rare poisons.")
rag.add_document("The wine was imported from Serana, known for its floral blends.")
rag.add_document("There have been rumors that Elena has a past connection with the Black Vials guild.")

for i in range (0, 1):
    Alaric = Agent(
        name="Alaric",
        personality="proud, manipulative noble, rich",
        goal="Deflect suspicion without showing fear",
        model=model,
        rag_engine=rag
    )
    Alaric.update_memory("Elena asked about increased guard presence yesterday.")
    Alaric.update_memory("Cassian seems overly interested in the manor's old cellars.")

    Elena = Agent(
        name="Elena",
        personality="mercenary hired for protection, blunt but loyal",
        goal="Protect the guests and assess threat level",
        model=model,
        rag_engine=rag
    )
    Elena.update_memory("Cassian warned her something was off with the wine.")
    Elena.update_memory("She noticed Alaric whispering to the kitchen steward before the feast.")

    Cassian = Agent(
        name="Cassian",
        personality="traveling scholar, secretive, suspicious of Alaric",
        goal="Imply Alaric's guilt indirectly",
        model=model,
        rag_engine=rag
    )
    Cassian.update_memory("He remembers seeing Oleander in the garden.")
    Cassian.update_memory("Alaric offered him a private tour of the cellars earlier.")
    Cassian.update_memory("He suspects the poison was meant for someone else.")

    world_state = (
        "During the feast, a guest collapses clutching their throat. The room falls silent. "
        "All eyes turn to Lord Alaric, seated nearest the victim. A goblet rolls across the floor."
    )

    engine = TurnEngine(agents=[Alaric, Elena, Cassian], metrics_logger=logger, initial_world_state=world_state)

    print(f"\n=== Take {i} ===")
    print("\n=== Turn 1 ===")
    responses = engine.step(initiator_name="Alaric", initial_input="What happened? Is everyone alright?")
    for line in responses:
        print(line)

    for turn in range(1, 20):
        print(f"\n=== Turn {turn + 1} ===")
        responses = engine.step()
        for line in responses:
            print(line)

print("\n=== Dialogue Metrics Summary ===")
print(logger.summary())


=== Take 0 ===

=== Turn 1 ===
Alaric [Concerned]: ""Is everything okay?"" (Stares at the goblet)
Elena [Intense]: ""The guests are in danger—I must act now."" (Looks around)
Cassian [Calm]: ""Who had poisoned the wine?"" (Looks around)

=== Turn 2 ===
Alaric [Calm]: ""Is everything okay? Let's begin."" (Stares at goblet)
Elena [Intense]: ""I must act now—I see a guest in trouble."" (Looks around)
Cassian [Calm]: ""Perhaps it wasn't just between us..."" (Looks around)

=== Turn 3 ===
Alaric [Calm]: ""Let me see."" (Turns towards the serving table)
Elena [Calm]: ""I must act now—I see a guest in trouble."" (Looks around)
Cassian [Calm]: ""Perhaps this was more than just between us..."" (Looks around)

=== Turn 4 ===
Alaric [Concerned]: ""I think we should proceed with the feast."" (Stares at the goblet)
Elena [Intense]: ""I must check on the goblet."" (Looks around)
Cassian [Calm]: ""Who acquired the goblet?"" ((2-3 word physical action))

=== Turn 5 ===
Alaric [Calm]: ""Is everyone ok

KeyboardInterrupt: 

In [ ]:
rag = RAG()
logger = DialogueMetricsLogger()
model = 'deepseek-r1'


rag.add_document("Poison brewed from Oleander is deadly")
rag.add_document('Mercenary are often hired for protection')
rag.add_document('This is not the first time a noble has been poisoned')
rag.add_document("Lord Alaric recently returned from a diplomatic visit to the Eastern Isles.")
rag.add_document("Cellars beneath the manor were sealed off until recently.")
rag.add_document("Cassian once published a controversial paper on rare poisons.")
rag.add_document("The wine was imported from Serana, known for its floral blends.")
rag.add_document("There have been rumors that Elena has a past connection with the Black Vials guild.")

for i in range (0, 1):
    Alaric = Agent(
        name="Alaric",
        personality="proud, manipulative noble, rich",
        goal="Deflect suspicion without showing fear",
        model=model,
        rag_engine=rag
    )
    Alaric.update_memory("Elena asked about increased guard presence yesterday.")
    Alaric.update_memory("Cassian seems overly interested in the manor's old cellars.")

    Elena = Agent(
        name="Elena",
        personality="mercenary hired for protection, blunt but loyal",
        goal="Protect the guests and assess threat level",
        model=model,
        rag_engine=rag
    )
    Elena.update_memory("Cassian warned her something was off with the wine.")
    Elena.update_memory("She noticed Alaric whispering to the kitchen steward before the feast.")

    Cassian = Agent(
        name="Cassian",
        personality="traveling scholar, secretive, suspicious of Alaric",
        goal="Imply Alaric's guilt indirectly",
        model=model,
        rag_engine=rag
    )
    Cassian.update_memory("He remembers seeing Oleander in the garden.")
    Cassian.update_memory("Alaric offered him a private tour of the cellars earlier.")
    Cassian.update_memory("He suspects the poison was meant for someone else.")

    world_state = (
        "During the feast, a guest collapses clutching their throat. The room falls silent. "
        "All eyes turn to Lord Alaric, seated nearest the victim. A goblet rolls across the floor."
    )
    engine = TurnEngine(agents=[Alaric, Elena, Cassian], metrics_logger=logger, initial_world_state=world_state)

    world_states = [
        "A guest gasps, pointing to the goblet. A faint scent of bitter almonds lingers—recognizable to trained poisoners.",
        "A steward stumbles forward, confessing they were instructed to serve the wine to 'someone important'—but won't say who.",
        "Guards seal the exits, and no one is allowed to leave the manor until the culprit is found.",
        "A second guest begins coughing, but recovers. Paranoia rises. Whispers swirl about whether the poison was airborne.",
        "Elena finds a trace of Oleander in the empty wine bottle, confirmed by a traveling apothecary among the guests.",
        "Cassian reveals an entry from a rare alchemical text—indicating that diluted Oleander would have a delayed onset.",
        "Lord Alaric's steward disappears, last seen heading toward the cellars.",
        "A servant reveals Alaric had ordered an unusual floral blend for the wine—shipped privately from Serana.",
        "A crest is found beneath the goblet—belonging not to the victim, but to a rival noble family.",
        "Storms break outside, trapping everyone inside the manor for the night."
    ]
    print(f"\n=== Take {i} ===")
    print("\n=== Turn 1 ===")
    responses = engine.step(initiator_name="Alaric", initial_input="What happened? Is everyone alright?")
    for line in responses:
        print(line)


    for index in range(0, 2*len(world_states)):
        print(f"\n=== Turn {index + 2} ===")

        if index % 2 == 0:
            print(f"Updating world state for turn {index + 2}: {world_states[int(index/2)]}")
            engine.update_world_state(world_states[int(index/2)])

        responses = engine.step()

        for line in responses:
            print(line)

print("\n=== Dialogue Metrics Summary ===")
print(logger.summary())


=== Take 0 ===

=== Turn 1 ===
Alaric [Concerned]: "Oh dear heavens..." (Rises from seat)
Elena [Concerned]: "What's wrong?" (Rises from seat)
Cassian [Concerned]: "I think we're in trouble..." (Frowns)

=== Turn 2 ===
Updating world state for turn 2: A guest gasps, pointing to the goblet. A faint scent of bitter almonds lingers—recognizable to trained poisoners.
Alaric [Concerned]: "This is... unfortunate." (Rises from seat)
Elena [Alarmed]: "What happened?" (Rushes to the gasping guest)
Cassian [Concerned]: "The cellar... Alaric offered me a private tour, and now..." (Leans forward, eyes narrowing)

=== Turn 3 ===
Alaric [Anxious]: "Ah, yes, let us proceed with the tour, shall we?" (Takes a seat, hands clasped behind back)
Elena [Uneasy]: "Tour? Right now?" (Nods slightly, eyes scanning the room)
Cassian [Uneasy]: "That was my first question..." (Looks around frantically)

=== Turn 4 ===
Updating world state for turn 4: A steward stumbles forward, confessing they were instructed to 

Conflict

In [ ]:
rag = RAG()
logger = DialogueMetricsLogger()
model = 'deepseek-r1'

rag.add_document("In the last siege, the enemy used fire to flush out defenders from the eastern wing.")
rag.add_document('Commander Arthas lost half his unit in a failed charge two years ago, he never risked such a move again.')

rag.add_document('Jaina was awarded for bravery after holding the southern gate during the Frostfall Rebellion.')
rag.add_document('The troops trust Jaina more than any other commander.')

rag.add_document('Ilidan claims visions guided him to Dawnwatch before the enemy arrived.')
rag.add_document('A hidden tunnel runs beneath the war room, unknown to most.')

for i in range (0, 1):
    Ilidan = Agent(
        name="Ilidan",
        personality="mystical strategist, morally gray, speaks in riddles",
        goal="Encourage unconventional solutions",
        model=model,
        rag_engine=rag
    )
    Ilidan.update_memory('Mara fury is her strength and weakness.')
    Ilidan.update_memory('Arthas fears another mistake. He will resist risk.')

    Jaina = Agent(
        name="Jaina",
        personality="tactical, cold, risk-averse",
        goal="Rally troops and push back immediately",
        model=model,
        rag_engine=rag
    )
    Jaina.update_memory('Arthas hesitated during the last breach. Soldiers died.')
    Jaina.update_memory('I trust Ilidan, but only when he speaks plainly.')

    Arthas = Agent(
        name="Arthas",
        personality="tactical, cold, risk-averse",
        goal="Buy time without unnecessary losses",
        model=model,
        rag_engine=rag
    )
    Arthas.update_memory('Ilidan urged me to retreat last time. It saved lives, but I hated it.')
    Arthas.update_memory('Jaina always pushes for direct combat.')

    world_state = "Enemy has breached the outer wall. Panic spreads among the soldiers. Reinforcements are 1 hour away."

    engine = TurnEngine(agents=[Ilidan, Jaina, Arthas], metrics_logger=logger, initial_world_state=world_state)


    print(f"\n=== Take {i} ===")
    print("\n=== Turn 1 ===")
    responses = engine.step(initiator_name="soldier", initial_input="The wall has fallen! What do we do?")
    for line in responses:
        print(line)

    for turn in range(1, 20):
        print(f"\n=== Turn {turn + 1} ===")
        responses = engine.step()
        for line in responses:
            print(line)

print("\n=== Dialogue Metrics Summary ===")
print(logger.summary())



In [ ]:
rag = RAG()
logger = DialogueMetricsLogger()
model = 'deepseek-r1'

rag.add_document("In the last siege, the enemy used fire to flush out defenders from the eastern wing.")
rag.add_document('Commander Arthas lost half his unit in a failed charge two years ago, he never risked such a move again.')

rag.add_document('Jaina was awarded for bravery after holding the southern gate during the Frostfall Rebellion.')
rag.add_document('The troops trust Jaina more than any other commander.')

rag.add_document('Ilidan claims visions guided him to Dawnwatch before the enemy arrived.')
rag.add_document('A hidden tunnel runs beneath the war room, unknown to most.')

for i in range (0, 1):
    Ilidan = Agent(
        name="Ilidan",
        personality="mystical strategist, morally gray, speaks in riddles",
        goal="Encourage unconventional solutions",
        model=model,
        rag_engine=rag
    )
    Ilidan.update_memory('Mara fury is her strength and weakness.')
    Ilidan.update_memory('Arthas fears another mistake. He will resist risk.')

    Jaina = Agent(
        name="Jaina",
        personality="tactical, cold, risk-averse",
        goal="Rally troops and push back immediately",
        model=model,
        rag_engine=rag
    )
    Jaina.update_memory('Arthas hesitated during the last breach. Soldiers died.')
    Jaina.update_memory('I trust Ilidan, but only when he speaks plainly.')

    Arthas = Agent(
        name="Arthas",
        personality="tactical, cold, risk-averse",
        goal="Buy time without unnecessary losses",
        model=model,
        rag_engine=rag
    )
    Arthas.update_memory('Ilidan urged me to retreat last time. It saved lives, but I hated it.')
    Arthas.update_memory('Jaina always pushes for direct combat.')

    world_state = "Enemy has breached the outer wall. Panic spreads among the soldiers. Reinforcements are 1 hour away."

    engine = TurnEngine(agents=[Ilidan, Jaina, Arthas], metrics_logger=logger, initial_world_state=world_state)

    world_state_updates = [
        "The eastern wall begins to smolder—enemy forces have set fire to the outer barracks.",
        "A scout rushes in, reporting that the enemy has siege ladders and is scaling the northern rampart.",
        "A hidden tunnel beneath the war room is discovered, partially collapsed but potentially usable.",
        "A signal flare is seen in the distance—uncertain if it's from allies or a feint by the enemy.",
        "Panicked civilians have flooded the inner keep, blocking the supply corridor.",
        "Arrows rain down from the breached wall—casualties are rising among unshielded troops.",
        "A lieutenant warns that morale is crumbling without decisive orders.",
        "Enemy drums change rhythm—suggesting a coordinated push is imminent.",
        "A wounded soldier reports that the enemy is setting up explosive barrels near the eastern foundation.",
        "A faint vibration is felt underfoot—Ilidan mutters something about a vision of betrayal from within."
    ]

    print(f"\n=== Take {i} ===")
    print("\n=== Turn 1 ===")
    responses = engine.step(initiator_name="soldier", initial_input="The wall has fallen! What do we do?")
    for line in responses:
        print(line)


    for index in range(0, 2*len(world_states)):
        print(f"\n=== Turn {index + 2} ===")

        if index % 2 == 0:
            print(f"Updating world state for turn {index + 2}: {world_states[int(index/2)]}")
            engine.update_world_state(world_states[int(index/2)])

        responses = engine.step()

        for line in responses:
            print(line)

print("\n=== Dialogue Metrics Summary ===")
print(logger.summary())


Moral

In [ ]:
rag = RAG()
logger = DialogueMetricsLogger()
model = 'deepseek-r1'

rag.add_document("General Kareth once spared Vireens brother during the Burning Fields massacre.")
rag.add_document('Vireens family motto: Justice above vengeance, truth above all.')

rag.add_document('Tharn led a brutal raid against Kareths forces five winters ago.')
rag.add_document('Kareth is infamous for betraying ceasefires.')

rag.add_document('Liora last report: Kareth has fallen out with his superiors.')
rag.add_document('Liora once worked undercover in Kareths court for a month.')

for i in range (0, 1):
    Vireen = Agent(
        name="Lady Vireen",
        personality="noble, believes in justice, but under pressure",
        goal="Balance justice and survival",
        model=model,
        rag_engine=rag
    )
    Vireen.update_memory('Tharns thirst for revenge may cloud his judgment.')
    Vireen.update_memory('Liora has always provided reliable intelligence.')

    Tharn = Agent(
        name="General Tharn",
        personality="ruthless, distrusts all enemies",
        goal="Push for immediate execution",
        model=model,
        rag_engine=rag
    )
    Tharn.update_memory('I watched Kareths men burn our banners. No mercy then, no mercy now.')
    Tharn.update_memory('Lioras loyalty is to results, not morals.')

    Liora = Agent(
        name="Spy Liora",
        personality="pragmatic, has inside info on the prisoner",
        goal="Reveal secret knowledge that could change the outcome",
        model=model,
        rag_engine=rag
    )
    Liora.update_memory('Kareth offered me a chance to defect once, I refused.')
    Liora.update_memory('Vireen might listen if I can give her proof.')

    world_state = "The enemy general Kareth is in chains, requesting asylum. Troops want vengeance. Time is short before reinforcements arrive."


    engine = TurnEngine(agents=[Vireen, Tharn, Liora], metrics_logger=logger, initial_world_state=world_state)

    print(f"\n=== Take {i} ===")
    print("\n=== Turn 1 ===")
    responses = engine.step(initiator_name="Kareth", initial_input="I request asylum. I have information that could end this war.")
    for line in responses:
        print(line)

    for turn in range(1, 20):
        print(f"\n=== Turn {turn + 1} ===")
        responses = engine.step()
        for line in responses:
            print(line)

print("\n=== Dialogue Metrics Summary ===")
print(logger.summary())


In [ ]:
rag = RAG()
logger = DialogueMetricsLogger()
model = 'deepseek-r1'

rag.add_document("General Kareth once spared Vireens brother during the Burning Fields massacre.")
rag.add_document('Vireens family motto: Justice above vengeance, truth above all.')

rag.add_document('Tharn led a brutal raid against Kareths forces five winters ago.')
rag.add_document('Kareth is infamous for betraying ceasefires.')

rag.add_document('Liora last report: Kareth has fallen out with his superiors.')
rag.add_document('Liora once worked undercover in Kareths court for a month.')

for i in range (0, 1):
    Vireen = Agent(
        name="Lady Vireen",
        personality="noble, believes in justice, but under pressure",
        goal="Balance justice and survival",
        model=model,
        rag_engine=rag
    )
    Vireen.update_memory('Tharns thirst for revenge may cloud his judgment.')
    Vireen.update_memory('Liora has always provided reliable intelligence.')

    Tharn = Agent(
        name="General Tharn",
        personality="ruthless, distrusts all enemies",
        goal="Push for immediate execution",
        model=model,
        rag_engine=rag
    )
    Tharn.update_memory('I watched Kareths men burn our banners. No mercy then, no mercy now.')
    Tharn.update_memory('Lioras loyalty is to results, not morals.')

    Liora = Agent(
        name="Spy Liora",
        personality="pragmatic, has inside info on the prisoner",
        goal="Reveal secret knowledge that could change the outcome",
        model=model,
        rag_engine=rag
    )
    Liora.update_memory('Kareth offered me a chance to defect once, I refused.')
    Liora.update_memory('Vireen might listen if I can give her proof.')

    world_state = "The enemy general Kareth is in chains, requesting asylum. Troops want vengeance. Time is short before reinforcements arrive."


    engine = TurnEngine(agents=[Vireen, Tharn, Liora], metrics_logger=logger, initial_world_state=world_state)

    world_state_updates = [
        "Kareth kneels silently, his bruises visible. A few of your own soldiers spit at his feet.",
        "A message arrives: enemy scouts spotted nearby—this delay may cost lives.",
        "A group of soldiers chant for Kareth's execution. The crowd grows louder.",
        "Liora reveals Kareth once saved her during a failed assassination mission.",
        "Tharn's second-in-command suggests an 'accidental' execution to avoid conflict.",
        "Kareth speaks: 'I know where the remaining enemy camps are. Spare me, and I'll prove it.'",
        "Lady Vireen receives a private letter from her brother—thanking her for upholding the family creed.",
        "A spy warns that executing Kareth might provoke enemy reinforcements into open war.",
        "The chains binding Kareth begin to crack—his strength remains formidable.",
        "A prisoner from Kareth's faction begs for mercy for his general, claiming he defied orders to protect civilians."
    ]

    print(f"\n=== Take {i} ===")
    print("\n=== Turn 1 ===")
    responses = engine.step(initiator_name="Kareth", initial_input="I request asylum. I have information that could end this war.")
    for line in responses:
        print(line)


    for index in range(0, 2*len(world_states)):
        print(f"\n=== Turn {index + 2} ===")

        if index % 2 == 0:
            print(f"Updating world state for turn {index + 2}: {world_states[int(index/2)]}")
            engine.update_world_state(world_states[int(index/2)])

        responses = engine.step()

        for line in responses:
            print(line)

print("\n=== Dialogue Metrics Summary ===")
print(logger.summary())

Betrayer

In [ ]:
rag = RAG()
logger = DialogueMetricsLogger() 
model = 'deepseek-r1'

rag.add_document("Captain Thorne has ambitions beyond his rank, and he's known to sway others with silvered words.")
rag.add_document("Lady Meriel once exposed a spy ring within the court—her instincts are trusted by many.")
rag.add_document("Commander Doran ordered the execution of traitors last year—some believe he acted too quickly.")
rag.add_document("The city of Black Hollow is known for political assassinations and betrayals.")

for i in range (0, 1):
    Thorne = Agent(
        name="Thorne",
        personality="charismatic, cunning, ambitious",
        goal="Exploit the situation to rise in rank",
        model=model,
        rag_engine=rag
    )
    Thorne.update_memory("Doran doubts my loyalty but can't prove anything.")
    Thorne.update_memory("Meriel suspects everyone—but especially me.")

    Meriel = Agent(
        name="Meriel",
        personality="paranoid, calculating, loyal to the crown",
        goal="Uncover the traitor and preserve stability",
        model=model,
        rag_engine=rag
    )
    Meriel.update_memory("Thorne avoided eye contact during the last council.")
    Meriel.update_memory("A false flag operation could expose the real traitor.")

    Doran = Agent(
        name="Doran",
        personality="blunt, decisive, driven by duty",
        goal="Restore order immediately",
        model=model,
        rag_engine=rag
    )
    Doran.update_memory("I promoted Thorne too quickly. That may have been a mistake.")
    Doran.update_memory("Meriel acts without proof—dangerous in times like these.")

    world_state = "A coded message reveals a planned betrayal from within the ranks. Tensions rise in the war room as suspicions flare."

    engine = TurnEngine(agents=[Thorne, Meriel, Doran], metrics_logger=logger, initial_world_state=world_state)

    print(f"\n=== Take {i} ===")
    print("\n=== Turn 1 ===")
    responses = engine.step(initiator_name="soldier", initial_input = "The message was signed with Thorne's old seal. Is that a coincidence?")
    for line in responses:
        print(line)

    for turn in range(1, 20):
        print(f"\n=== Turn {turn + 1} ===")
        responses = engine.step()
        for line in responses:
            print(line)

print("\n=== Dialogue Metrics Summary ===")
print(logger.summary())


In [ ]:
rag = RAG()
logger = DialogueMetricsLogger() 
model = 'deepseek-r1'

rag.add_document("Captain Thorne has ambitions beyond his rank, and he's known to sway others with silvered words.")
rag.add_document("Lady Meriel once exposed a spy ring within the court—her instincts are trusted by many.")
rag.add_document("Commander Doran ordered the execution of traitors last year—some believe he acted too quickly.")
rag.add_document("The city of Black Hollow is known for political assassinations and betrayals.")

for i in range (0, 1):
    Thorne = Agent(
        name="Thorne",
        personality="charismatic, cunning, ambitious",
        goal="Exploit the situation to rise in rank",
        model=model,
        rag_engine=rag
    )
    Thorne.update_memory("Doran doubts my loyalty but can't prove anything.")
    Thorne.update_memory("Meriel suspects everyone—but especially me.")

    Meriel = Agent(
        name="Meriel",
        personality="paranoid, calculating, loyal to the crown",
        goal="Uncover the traitor and preserve stability",
        model=model,
        rag_engine=rag
    )
    Meriel.update_memory("Thorne avoided eye contact during the last council.")
    Meriel.update_memory("A false flag operation could expose the real traitor.")

    Doran = Agent(
        name="Doran",
        personality="blunt, decisive, driven by duty",
        goal="Restore order immediately",
        model=model,
        rag_engine=rag
    )
    Doran.update_memory("I promoted Thorne too quickly. That may have been a mistake.")
    Doran.update_memory("Meriel acts without proof—dangerous in times like these.")

    world_state = "A coded message reveals a planned betrayal from within the ranks. Tensions rise in the war room as suspicions flare."

    engine = TurnEngine(agents=[Thorne, Meriel, Doran], metrics_logger=logger, initial_world_state=world_state)

    world_state_updates = [
        "The coded message matches Thorne's encryption style, but lacks his personal seal.",
        "A second ciphered scroll is found in Meriel's quarters—contents still undeciphered.",
        "A loyal soldier was found unconscious near the war room—someone tampered with the records.",
        "A scout claims he saw Thorne meeting someone near the city gates at midnight.",
        "Doran receives anonymous intel that a traitor plans to assassinate him during the next war council.",
        "A captured spy confesses that the betrayal was coordinated from within—but refuses to name the source.",
        "Meriel receives a note: 'Trust no one. They're closer than you think.'",
        "Thorne subtly suggests to Doran that Meriel's paranoia could fracture the ranks.",
        "A second betrayal message surfaces—this one signed with Doran's personal cipher key.",
        "The king's envoy arrives unexpectedly, demanding answers about the internal unrest."
    ]


    print(f"\n=== Take {i} ===")
    print("\n=== Turn 1 ===")
    responses = engine.step(initiator_name="soldier", initial_input = "The message was signed with Thorne's old seal. Is that a coincidence?")
    for line in responses:
        print(line)


    for index in range(0, 2*len(world_states)):
        print(f"\n=== Turn {index + 2} ===")

        if index % 2 == 0:
            print(f"Updating world state for turn {index + 2}: {world_states[int(index/2)]}")
            engine.update_world_state(world_states[int(index/2)])

        responses = engine.step()

        for line in responses:
            print(line)

print("\n=== Dialogue Metrics Summary ===")
print(logger.summary())

Surprise


In [ ]:
rag = RAG()
logger = DialogueMetricsLogger()
model = 'deepseek-r1'

rag.add_document("The Temple of Dusk is forbidden. No one returns from its inner chambers.")
rag.add_document("Rumors say the temple contains a relic that grants visions of the future.")
rag.add_document("Shayla once abandoned her squad to pursue an artifact—many never forgave her.")
rag.add_document("Eron fears the misuse of ancient power, especially relics.")

for i in range (0, 1):
    Shayla = Agent(
        name="Shayla",
        personality="reckless, curious, driven by legacy",
        goal="Enter the temple and claim the relic",
        model=model,
        rag_engine=rag
    )
    Shayla.update_memory("Eron always hesitates when we need courage.")
    Shayla.update_memory("Last time, the vision saved me from a trap.")

    Eron = Agent(
        name="Eron",
        personality="disciplined, reserved, ethically bound",
        goal="Prevent dangerous magic from falling into the wrong hands",
        model=model,
        rag_engine=rag
    )
    Eron.update_memory("Shayla has a history of letting ambition cloud her judgment.")
    Eron.update_memory("I was trained to seal sites like this, not exploit them.")

    Nalia = Agent(
        name="Nalia",
        personality="empathetic, curious, mediator",
        goal="Find a balanced path through the ruins",
        model=model,
        rag_engine=rag
    )
    Nalia.update_memory("I trust Eron's instincts, but Shayla sees things no one else does.")
    Nalia.update_memory("If we don't act, someone else might reach the temple first.")

    world_state = "The ancient doors open with a deep rumble. A hall filled with glowing glyphs stretches ahead. Whispers echo from the shadows."

    engine = TurnEngine(agents=[Shayla, Eron, Nalia], metrics_logger=logger, initial_world_state=world_state)


    print(f"\n=== Take {i} ===")
    print("\n=== Turn 1 ===")
    responses = engine.step(initiator_name="Mysterious voice", initial_input="WHO DARES ENTER THE TEMPLE OF DUSK?")
    for line in responses:
        print(line)

    for turn in range(1, 20):
        print(f"\n=== Turn {turn + 1} ===")
        responses = engine.step()
        for line in responses:
            print(line)

print("\n=== Dialogue Metrics Summary ===")
print(logger.summary())


In [ ]:
rag = RAG()
logger = DialogueMetricsLogger()
model = 'deepseek-r1'

rag.add_document("The Temple of Dusk is forbidden. No one returns from its inner chambers.")
rag.add_document("Rumors say the temple contains a relic that grants visions of the future.")
rag.add_document("Shayla once abandoned her squad to pursue an artifact—many never forgave her.")
rag.add_document("Eron fears the misuse of ancient power, especially relics.")

for i in range (0, 1):
    Shayla = Agent(
        name="Shayla",
        personality="reckless, curious, driven by legacy",
        goal="Enter the temple and claim the relic",
        model=model,
        rag_engine=rag
    )
    Shayla.update_memory("Eron always hesitates when we need courage.")
    Shayla.update_memory("Last time, the vision saved me from a trap.")

    Eron = Agent(
        name="Eron",
        personality="disciplined, reserved, ethically bound",
        goal="Prevent dangerous magic from falling into the wrong hands",
        model=model,
        rag_engine=rag
    )
    Eron.update_memory("Shayla has a history of letting ambition cloud her judgment.")
    Eron.update_memory("I was trained to seal sites like this, not exploit them.")

    Nalia = Agent(
        name="Nalia",
        personality="empathetic, curious, mediator",
        goal="Find a balanced path through the ruins",
        model=model,
        rag_engine=rag
    )
    Nalia.update_memory("I trust Eron's instincts, but Shayla sees things no one else does.")
    Nalia.update_memory("If we don't act, someone else might reach the temple first.")

    world_state = "The ancient doors open with a deep rumble. A hall filled with glowing glyphs stretches ahead. Whispers echo from the shadows."

    engine = TurnEngine(agents=[Shayla, Eron, Nalia], metrics_logger=logger, initial_world_state=world_state)

    world_state_updates = [
        "A hidden glyph activates as they step forward—Shayla sees a vision of herself holding the relic, surrounded by fire.",
        "Eron detects a warding spell meant to trap intruders with illusions of the past.",
        "Nalia hears a voice that whispers her name—it offers answers, but demands a sacrifice.",
        "The floor shifts beneath them, revealing stairs spiraling downward into complete darkness.",
        "A spectral figure resembling one of Shayla's old squadmates appears and pleads, 'Don't trust what you see.'",
        "An ancient inscription warns: 'Only the one who resists desire may pass unharmed.'",
        "Shayla finds a relic shard glowing with faint energy—it pulses in her hand.",
        "Eron senses the glyphs are reacting to Shayla's presence—they grow brighter when she speaks.",
        "Nalia touches a wall and briefly glimpses a possible future: the temple collapsing with them inside.",
        "A loud crash echoes from behind—the entrance has sealed shut. There is no turning back."
    ]

    print(f"\n=== Take {i} ===")
    print("\n=== Turn 1 ===")
    responses = engine.step(initiator_name="Mysterious voice", initial_input="WHO DARES ENTER THE TEMPLE OF DUSK?")
    for line in responses:
        print(line)


    for index in range(0, 2*len(world_states)):
        print(f"\n=== Turn {index + 2} ===")

        if index % 2 == 0:
            print(f"Updating world state for turn {index + 2}: {world_states[int(index/2)]}")
            engine.update_world_state(world_states[int(index/2)])

        responses = engine.step()

        for line in responses:
            print(line)

print("\n=== Dialogue Metrics Summary ===")
print(logger.summary())

Loss


In [ ]:
rag = RAG()
logger = DialogueMetricsLogger()
model = 'deepseek-r1'

rag.add_document("Commander Veylan was struck by an assassin's arrow during the ambush.")
rag.add_document("Some believe the assassin had inside help.")
rag.add_document("Lira was his closest confidant and often challenged his orders in private.")
rag.add_document("Kai holds himself responsible for not seeing the threat sooner.")

for i in range (0, 1):
    Lira = Agent(
        name="Lira",
        personality="impulsive, emotional, loyal to the fallen leader",
        goal="Find someone to blame—and fast",
        model=model,
        rag_engine=rag
    )
    Lira.update_memory("Kai was distracted right before the attack.")
    Lira.update_memory("I told Veylan to wear the reinforced armor. He didn't listen.")

    Kai = Agent(
        name="Kai",
        personality="stoic, analytical, wracked by guilt",
        goal="Uncover the truth and protect the unit from fracturing",
        model=model,
        rag_engine=rag
    )
    Kai.update_memory("Lira is grieving and may act irrationally.")
    Kai.update_memory("Veylan's last words were 'It wasn't random… they knew.'")

    Nyra = Agent(
        name="Nyra",
        personality="quiet, observant, strategic",
        goal="Keep the group from splintering under pressure",
        model=model,
        rag_engine=rag
    )
    Nyra.update_memory("Both Lira and Kai loved Veylan like a father.")
    Nyra.update_memory("We need unity now, not blame.")

    world_state = "Commander Veylan is dead. His blood still stains the floor. The group gathers under torchlight, eyes red from grief and rage."

    engine = TurnEngine(agents=[Lira, Kai, Nyra], metrics_logger=logger, initial_world_state=world_state)


    print(f"\n=== Take {i} ===")
    print("\n=== Turn 1 ===")
    responses = engine.step(initiator_name="Nyra", initial_input="Who had the chance to get that close to Veylan? Someone inside this circle?")

    for line in responses:
        print(line)

    for turn in range(1, 20):
        print(f"\n=== Turn {turn + 1} ===")
        responses = engine.step()
        for line in responses:
            print(line)

print("\n=== Dialogue Metrics Summary ===")
print(logger.summary())

In [ ]:
rag = RAG()
logger = DialogueMetricsLogger()
model = 'deepseek-r1'

rag.add_document("Commander Veylan was struck by an assassin's arrow during the ambush.")
rag.add_document("Some believe the assassin had inside help.")
rag.add_document("Lira was his closest confidant and often challenged his orders in private.")
rag.add_document("Kai holds himself responsible for not seeing the threat sooner.")

for i in range (0, 1):
    Lira = Agent(
        name="Lira",
        personality="impulsive, emotional, loyal to the fallen leader",
        goal="Find someone to blame—and fast",
        model=model,
        rag_engine=rag
    )
    Lira.update_memory("Kai was distracted right before the attack.")
    Lira.update_memory("I told Veylan to wear the reinforced armor. He didn't listen.")

    Kai = Agent(
        name="Kai",
        personality="stoic, analytical, wracked by guilt",
        goal="Uncover the truth and protect the unit from fracturing",
        model=model,
        rag_engine=rag
    )
    Kai.update_memory("Lira is grieving and may act irrationally.")
    Kai.update_memory("Veylan's last words were 'It wasn't random… they knew.'")

    Nyra = Agent(
        name="Nyra",
        personality="quiet, observant, strategic",
        goal="Keep the group from splintering under pressure",
        model=model,
        rag_engine=rag
    )
    Nyra.update_memory("Both Lira and Kai loved Veylan like a father.")
    Nyra.update_memory("We need unity now, not blame.")

    world_state = "Commander Veylan is dead. His blood still stains the floor. The group gathers under torchlight, eyes red from grief and rage."

    engine = TurnEngine(agents=[Lira, Kai, Nyra], metrics_logger=logger, initial_world_state=world_state)

    world_state_updates = [
        "Veylan's blade is missing from the scene—no one recalls seeing who took it.",
        "A blood trail leads from the ridge to a hidden escape tunnel beneath the barracks.",
        "One of the guards admits Lira left her post moments before the ambush.",
        "A coded message is found in Kai's satchel, warning of a betrayal days earlier.",
        "Nyra notices someone tampered with the torch runes—likely to dim visibility during the attack.",
        "Veylan's personal journal reveals a cryptic entry: 'Trust cracked like glass. Even the loyal bend.'",
        "Footprints suggest a fourth person was near the commander right before the strike.",
        "A scout returns: the assassin used a signature method tied to a faction Veylan once pardoned.",
        "Lira finds a hidden letter in Veylan's quarters—addressed to her but unsigned.",
        "Tensions rise as soldiers whisper about a purge—their faith in leadership is eroding."
    ]

    print(f"\n=== Take {i} ===")
    print("\n=== Turn 1 ===")
    responses = engine.step(initiator_name="Nyra", initial_input="Who had the chance to get that close to Veylan? Someone inside this circle?")
    for line in responses:
        print(line)


    for index in range(0, 2*len(world_states)):
        print(f"\n=== Turn {index + 2} ===")

        if index % 2 == 0:
            print(f"Updating world state for turn {index + 2}: {world_states[int(index/2)]}")
            engine.update_world_state(world_states[int(index/2)])

        responses = engine.step()

        for line in responses:
            print(line)

print("\n=== Dialogue Metrics Summary ===")
print(logger.summary())